In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import os 
import time
from datetime import timedelta

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import itertools
from PIL import Image
import shutil

import pandas as pd
import numpy as np

In [2]:
tran_filenames = os.listdir('data/train')
datasetdir = os.path.join('./data')
traindir = os.path.join(datasetdir,'train2')
testdir = os.path.join(datasetdir,'test2')

In [3]:
train_cat = filter(lambda x:x[:3]=='cat', tran_filenames)
train_dog = filter(lambda x:x[:3]=='dog', tran_filenames)

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
def feature_extract(model, image_size, imagedir, save_file):
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
    dataset = datasets.ImageFolder(imagedir, transforms.Compose([
        transforms.RandomResizedCrop(image_size),  #224
        transforms.RandomHorizontalFlip(),  # 随机挑一些图镜像翻转
        transforms.ToTensor(),
        normalize,]))
    
    batch_size = 32
    
    loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=10,
        # drop_last=True
    )
    
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    model = model.to(device)
    model.eval() 
    x_extract = []
    y_extract = []
    for x, y in loader:
        with torch.no_grad():
            x = x.cuda()
            x_ext = model(x)
            x_ext = F.adaptive_avg_pool2d(x_ext, (1,1))
            x_extract.append(x_ext.data.cpu().numpy())  # 只有一个Tensor可以转换成numpy
            y_extract.append(y.data.numpy())
    x_nobatch_ext = np.array([x for item in x_extract for x in item])
    y_nobatch_ext = np.array([x for item in y_extract for x in item])
    print("x shape:", x_nobatch_ext.shape)
    print("y sahpe:", y_nobatch_ext.shape)
    np.save(save_file, x_nobatch_ext)
    np.save("y-{}".format(save_file), y_nobatch_ext)
    print("Done!")

In [6]:
models_dic = {}
model1 = nn.Sequential(*list(models.resnet50(pretrained=True).children())[:-2])  # 这里删除最后两层为了和后面的模型统一
models_dic["resnet50"] = model1
model2 = nn.Sequential(*list(models.densenet161(pretrained=True).children())[:-1]) 
models_dic["densenet161"] = model2
# model3 = nn.Sequential(*list(models.vgg19(pretrained=True).children())[:-1])
# models_dic["vgg19"] = model3
model4 = nn.Sequential(*list(models.resnet152(pretrained=True).children())[:-2]) 
models_dic["resnet152"] = model4
image_dir = {
    "train_dir":traindir, 
    "test_dir":testdir
}

In [7]:
image_size = 224
for m_name, m in models_dic.items():
    for d_name,d in image_dir.items():
        save_file = "{}-{}".format(m_name, d_name)
        print("SaveFile:", save_file)
        feature_extract(m, image_size=image_size, imagedir=d, save_file=save_file)
print("All Done!")

SaveFile: resnet152-train_dir
x shape: (25000, 2048, 1, 1)
y sahpe: (25000,)
Done!
SaveFile: resnet152-test_dir
x shape: (12500, 2048, 1, 1)
y sahpe: (12500,)
Done!
All Done!


In [8]:
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout

Using TensorFlow backend.


In [9]:
x_test_li = ["densenet161-test_dir.npy", "resnet50-test_dir.npy","resnet152-test_dir.npy"]
x_train_li = ["densenet161-train_dir.npy","resnet50-train_dir.npy","resnet152-train_dir.npy"]
y_train_li = ["y-densenet161-train_dir.npy", "y-resnet50-train_dir.npy", "y-resnet152-train_dir.npy"]

In [10]:
def creat_dataset(npfiles):
    concat_fea = []
    for f in npfiles:
        np_load = np.load(f)
        concat_fea.append(np_load)
    concat_fea = np.concatenate(concat_fea, axis=1)
    return concat_fea

In [11]:
X = creat_dataset(x_train_li)

In [12]:
X_train = X.reshape(X.shape[0],X.shape[1])

In [13]:
X_t= creat_dataset(x_test_li)

In [14]:
X_test = X_t.reshape(X_t.shape[0], X_t.shape[1])

In [15]:
X_test.shape

(12500, 6304)

In [16]:
y_train = np.load(y_train_li[0])

In [17]:
y_train.shape

(25000,)

In [18]:
from sklearn.utils import shuffle

In [19]:
X_train, y_train = shuffle(X_train, y_train)

In [21]:
np.random.seed(2019)

In [22]:
def create_model(X_train):
    input_tensor = keras.Input(X_train.shape[1:])
    x = Dropout(0.5)(input_tensor)
    x = Dense(1, activation='sigmoid')(x)
    model = keras.Model(input_tensor, x)
    model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [23]:
model = create_model(X_train)

In [24]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 6304)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 6304)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6305      
Total params: 6,305
Trainable params: 6,305
Non-trainable params: 0
_________________________________________________________________


In [25]:
hist = model.fit(X_train, y_train, batch_size=128, epochs=8, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/8
20000/20000 [==============================] - 4s 207us/step - loss: 0.0903 - acc: 0.9669 - val_loss: 0.0238 - val_acc: 0.9928
Epoch 2/8
20000/20000 [==============================] - 2s 108us/step - loss: 0.0294 - acc: 0.9905 - val_loss: 0.0180 - val_acc: 0.9946
Epoch 3/8
20000/20000 [==============================] - 2s 110us/step - loss: 0.0235 - acc: 0.9927 - val_loss: 0.0179 - val_acc: 0.9946
Epoch 4/8
20000/20000 [==============================] - 2s 120us/step - loss: 0.0215 - acc: 0.9925 - val_loss: 0.0239 - val_acc: 0.9916
Epoch 5/8
20000/20000 [==============================] - 2s 113us/step - loss: 0.0211 - acc: 0.9928 - val_loss: 0.0190 - val_acc: 0.9938
Epoch 6/8
20000/20000 [==============================] - 2s 108us/step - loss: 0.0190 - acc: 0.9940 - val_loss: 0.0167 - val_acc: 0.9944
Epoch 7/8
20000/20000 [==============================] - 2s 114us/step - loss: 0.0184 - acc: 0.9942 - val_loss: 0.0165 - val_acc:

In [26]:
test_images = [os.path.join(testdir, 'test', f) for f in sorted(os.listdir(os.path.join(testdir,'test')))]

In [27]:
def save_csv(preds, images, csv_name, clib=False):
    if clib:
        preds = preds.clip(min=0.005,max=0.995)
    pred_result = preds[:, 0]
    results = zip(images, pred_result)
    idx = [(x[0].split('/')[-1]).split('.')[0] for x in results]
    res = pd.DataFrame.from_dict({
        'id': idx,
        'label': pred_result.tolist()
    })
    res = res.set_index('id')
    res.to_csv(csv_name)
    print("Save {} done.".format(csv_name))

In [28]:
preds = model.predict(X_test)

In [29]:
save_csv(preds, test_images, "submission.csv", False)

Save submission.csv done.


In [30]:
save_csv(preds, test_images, "submission-trick.csv", True)

Save submission-trick.csv done.


In [31]:
def test_feature_extract(model, image_size, imagedir, save_file):
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
    dataset = datasets.ImageFolder(imagedir, transforms.Compose([
        transforms.RandomResizedCrop(image_size),  #224
        transforms.RandomHorizontalFlip(),  # 随机挑一些图镜像翻转
        transforms.ToTensor(),
        normalize,]))
    
    batch_size = 32
    
    loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=10,
        # drop_last=True
    )
    
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    model = model.to(device)
    model.eval() 
    x_extract = []
    y_extract = []
    for x, y in loader:
        with torch.no_grad():
            x = x.cuda()
            x_ext = model(x)
            x_ext = F.adaptive_avg_pool2d(x_ext, (1,1))
            x_extract.append(x_ext.data.cpu().numpy())  # 只有一个Tensor可以转换成numpy
            y_extract.append(y.data.numpy())
            break
    x_nobatch_ext = np.array([x for item in x_extract for x in item])
    y_nobatch_ext = np.array([x for item in y_extract for x in item])
    print("x shape:", x_nobatch_ext.shape)
    print("y sahpe:", y_nobatch_ext.shape)
    print("Done!")

In [32]:
test_feature_extract(nn.Sequential(*list(models.resnet50(pretrained=True).children())[:-2]),
                image_size=224, imagedir=testdir, save_file="xxx") 

x shape: (32, 2048, 1, 1)
y sahpe: (32,)
Done!


In [33]:
# 在pytorch 里尝试把最后一层压扁没有成功
# https://zhuanlan.zhihu.com/p/25978105